In [1]:
import sys
sys.path.append('..')

import os
import psutil

import numpy as np
np.random.seed(0)

In [2]:
import matplotlib.pyplot as plt
from pylab import *
from keras.models import Sequential
from keras import optimizers
from keras.models import load_model
from keras.optimizers import Adam, SGD
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout, Convolution2D
from keras.models import Model
from keras.datasets import cifar10

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform,he_uniform

from keras.preprocessing.image import ImageDataGenerator
from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K
from keras.utils import plot_model,normalize

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from custom_lib.triplet_utils import buildDataSet, build_model
from custom_lib.triplet_utils import get_batch_hard, compute_probs
from custom_lib.triplet_utils import add_top, remove_top

from art.classifiers import KerasClassifier
from art.attacks.projected_gradient_descent import ProjectedGradientDescent
from art.attacks.iterative_method import BasicIterativeMethod
from art.defences.adversarial_trainer import AdversarialTrainer

from custom_lib.build_resnet import resnet_v1, resnet_emb

Using TensorFlow backend.


In [12]:
model = load_model("saved_models/cifar10_ResNet32v1_model.h5")

model.compile(optimizer=SGD(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [13]:
from keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train / 255.
x_test = x_test / 255.

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [14]:
def restrict_cpu():
    p = psutil.Process()

    for i in p.threads():
        temp = psutil.Process(i.id)

        temp.cpu_affinity([i for i in range(4)])

In [5]:
from art.attacks.fast_gradient import FastGradientMethod

art_model = KerasClassifier(clip_values=(0, 1.), model=model, use_logits=False)
attack = FastGradientMethod(art_model, eps=0.04, batch_size=64)

adv_imgs = attack.generate(x_test[0:10])

restrict_cpu()

adv_train = attack.generate(x_train)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [8]:
print(model.evaluate(x_test, y_test))
print(model.evaluate(adv_imgs, y_test))

10000/10000 [==============================] - 3s 277us/step
[0.3938062877178192, 0.9184]
10000/10000 [==============================] - 2s 189us/step
[4.849322650146484, 0.2105]


In [9]:
trainer = AdversarialTrainer(art_model, attack, ratio=1.0)
trainer.fit(x_train, y_train, nb_epochs=5, batch_size=32, verbose=0)

In [11]:
adv_imgs = attack.generate(x_test)

print(model.evaluate(x_test, y_test))
print(model.evaluate(adv_imgs, y_test))

10000/10000 [==============================] - 2s 202us/step
[1.3223604904174804, 0.6053]
10000/10000 [==============================] - 2s 192us/step
[1.6293663112640382, 0.4343]


In [42]:
diffs = abs(adv_imgs - x_test)
zeros = [i.sum() < 1e-2 for i in x_test]
sum(zeros)

0

In [8]:
model.fit(adv_train, y_train, validation_data=(x_test, y_test), batch_size=64, epochs=10)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 26s 523us/step - loss: 1.2708 - acc: 0.6106 - val_loss: 0.7560 - val_acc: 0.7905
Epoch 2/10
50000/50000 [==============================] - 25s 498us/step - loss: 1.0550 - acc: 0.6676 - val_loss: 0.7027 - val_acc: 0.8034
Epoch 3/10
50000/50000 [==============================] - 25s 500us/step - loss: 0.9441 - acc: 0.7013 - val_loss: 0.6673 - val_acc: 0.8135
Epoch 4/10
50000/50000 [==============================] - 25s 497us/step - loss: 0.8745 - acc: 0.7240 - val_loss: 0.6494 - val_acc: 0.8183
Epoch 5/10
50000/50000 [==============================] - 25s 497us/step - loss: 0.8212 - acc: 0.7432 - val_loss: 0.6272 - val_acc: 0.8257
Epoch 6/10
50000/50000 [==============================] - 25s 494us/step - loss: 0.7871 - acc: 0.7523 - val_loss: 0.6190 - val_acc: 0.8285
Epoch 7/10
50000/50000 [==============================] - 25s 492us/step - loss: 0.7576 - acc: 0.7662 - val_loss: 0.

In [9]:
adv_imgs = attack.generate(x_test)

print(model.evaluate(x_test, y_test))
print(model.evaluate(adv_imgs, y_test))

10000/10000 [==============================] - 2s 189us/step
[0.5895575842380524, 0.8394]
10000/10000 [==============================] - 2s 186us/step
[3.599535289001465, 0.214]


In [6]:
trainer = AdversarialTrainer(art_model, attack, ratio=0.5)
trainer.fit(x_train, y_train, nb_epochs=5, batch_size=32, verbose=0)

In [7]:
adv_imgs = attack.generate(x_test)

print(model.evaluate(x_test, y_test))
print(model.evaluate(adv_imgs, y_test))

10000/10000 [==============================] - 3s 309us/step
[0.41616658363342285, 0.9049]
10000/10000 [==============================] - 2s 190us/step
[1.519617756652832, 0.4683]
